### Apprentissage 

In [1]:
# Imports : keras 2.6.1 / tensorflow / theano / ViennaRNA package python (installer via 'conda install viennarna')

import keras
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.utils import np_utils, generic_utils
from keras.optimizers import SGD, RMSprop, Adadelta, Adagrad, Adam
from keras.layers.convolutional import Conv2D, MaxPooling2D,Conv1D, MaxPooling1D
from keras.models import model_from_config
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.models import load_model
from keras.layers.convolutional import AveragePooling2D
from keras.callbacks import EarlyStopping
import RNA
from sklearn.metrics import roc_curve, auc
import numpy as np

import random
import gzip
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from scipy import sparse
import pdb
from math import  sqrt
from sklearn.metrics import roc_curve, auc
import theano
import subprocess as sp
import scipy.stats as stats
import argparse
import pandas as pd
import csv
from random import shuffle

/home/louis_jeay/downloads/3-5-2-test/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [126]:
# Fonction de lecture des probabilités d'accessibilité après lecture en csv

def clean(seq):
    seq = seq.split('\n')
    seq2 = ''
    for j in seq:
        seq2 = seq2 + j
    seq2 = seq2[1:len(seq2)-1]
    seq2 = seq2.split(' ')
    #print(seq2)
    seq3=[]
    for j in seq2:
        #print(j)
        if j=='':
            a=0
        else:
            seq3.append(float(j))
    return seq3

# Fonction de chargement des données d'apprentissage et de benchmark

def load_data(nb_train = 134000,nb_val = 6000,nb_test = 25000, nb_benchmark=5000):
    # load your data using this function
    verif = []

    verif = pd.read_csv("genomes/human_divers_negatifsv2.csv", sep = "\t",header=None)
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")            
    neg1 = verif
    verif=[]

    verif = pd.read_csv("genomes/human_mRNA-mRNA_negatifsv2.csv", sep = "\t",header=None)
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
    neg2 = verif
    verif=[]
    verif = pd.read_csv("genomes/mouse_divers_negv2.csv", sep = "\t",header=None)
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
    neg3 = verif
    verif=[]
    verif = pd.read_csv("genomes/human_diversv2_neg_suppl.csv", sep = "\t",header=None)
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
    neg4 = verif
    verif=[]
    verif = pd.read_csv("genomes/human_mRNA-mRNA_negatifsv2_suppl.csv", sep = "\t",header=None)
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
    neg5 = verif
    verif=[]

    verif = pd.read_csv("genomes/human_diversv2.csv", sep = "\t",header=None)  
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            
    pos1 = verif
    l = len(pos1)
    verif=[]

    verif = pd.read_csv("genomes/human_mRNA-mRNAv2.csv", sep = "\t",header=None)  
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            
    pos2 = verif
    l=l+len(pos2)
    verif=[]

    verif = pd.read_csv("genomes/mouse_divers_posv2.csv", sep = "\t",header=None)  
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            
    pos3 = verif
    l=l+len(pos3)
    verif=[]

    bdd = np.concatenate((pos1,pos2,pos3,neg1,neg2,neg3,neg4,neg5))

    pos1 = []
    neg1 = []
    pos2 = []
    neg2 = []
    pos3 = []
    neg3 = []

    labels = np.zeros((len(bdd),1))

    bdd = np.concatenate((bdd,labels),axis=1)
    for i in range(l):
        bdd[i,4]=1
    labels=[]
    
    # shuffle total

    indices = np.arange(len(bdd))
    shuffle(indices)
    bdd = bdd[indices]
    indices=[]
    print(len(bdd))
    # 
    l = len(bdd) - nb_benchmark
    matrice = np.zeros((l,36,36,1)) 
    matrice2 = np.zeros((l,36,4))
    matrice3 = np.zeros((l,36,4))
    matrice4 = np.zeros((l,36,4))
    matrice5 = np.zeros((l,36,4))

    for i in range(l):
        seq1 = bdd[i,0][85:121]
        seq2 = bdd[i,1][85:121]
       
        for j in range(len(seq1)):
            for k in range(len(seq2)):
                if (seq1[j]=='a' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='a'):
                    matrice[i,j,k,0] = 1
                elif (seq1[j]=='g' and seq2[k]=='c') or (seq1[j]=='c' and seq2[k]=='g'):
                    matrice[i,j,k,0] = 1
                elif (seq1[j]=='g' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='g'):
                    matrice[i,j,k,0] = 1
        for j in range(len(seq1)):
            if seq1[j]=='a':
                matrice2[i,j,0] = 1
            elif seq1[j]=='u':
                matrice2[i,j,1] = 1
            elif seq1[j]=='g':
                matrice2[i,j,2] = 1
            elif seq1[j]=='c':
                matrice2[i,j,3] = 1
        for j in range(len(seq2)):
            if seq2[j]=='a':
                matrice3[i,j,0] = 1
            elif seq2[j]=='u':
                matrice3[i,j,1] = 1
            elif seq2[j]=='g':
                matrice3[i,j,2] = 1
            elif seq2[j]=='c':
                matrice3[i,j,3] = 1

    for i in range(36):
        matrice4[:,36-i-1,:] = matrice2[:,i,:]
    for i in range(36):
        matrice5[:,36-i-1,:] = matrice3[:,i,:]
    
    training = []
    training.append(matrice[:nb_train])
    training.append(matrice2[:nb_train])
    training.append(matrice4[:nb_train])
    training.append(matrice3[:nb_train])
    training.append(matrice5[:nb_train])    
    
    validation = []
    validation.append(matrice[nb_train:nb_train+nb_val])
    validation.append(matrice2[nb_train:nb_train+nb_val])
    validation.append(matrice4[nb_train:nb_train+nb_val])
    validation.append(matrice3[nb_train:nb_train+nb_val])
    validation.append(matrice5[nb_train:nb_train+nb_val])

    testing = []
    testing.append(matrice[-nb_test:])
    testing.append(matrice2[-nb_test:])
    testing.append(matrice4[-nb_test:])
    testing.append(matrice3[-nb_test:])
    testing.append(matrice5[-nb_test:])
    
    data_benchmark = bdd[-nb_benchmark:]
    
    labels = bdd[:l,4]
    bdd = []
    y = labels[:nb_train]
    y = keras.utils.np_utils.to_categorical(y,2)
    val_y = labels[nb_train:nb_train+nb_val]
    val_y = keras.utils.np_utils.to_categorical(val_y,2)
    true_y = labels[-nb_test:]

    return training, y, validation, val_y, testing, true_y, data_benchmark

In [3]:
# Calcul de performances pour l'apprentissage et le benchmark : acc et MCC
def calculate_performance(test_num, pred_y,  labels):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for index in range(test_num):
        if labels[index] ==1:
            if labels[index] == pred_y[index]:
                tp = tp +1
            else:
                fn = fn + 1
        else:
            if labels[index] == pred_y[index]:
                tn = tn +1
            else:
                fp = fp + 1               
            
    acc = float(tp + tn)/test_num
    precision = float(tp)/(tp+ fp)
    sensitivity = float(tp)/ (tp+fn)
    specificity = float(tn)/(tn + fp)
    MCC = float(tp*tn-fp*fn)/(np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)))
    return acc, precision, sensitivity, specificity, MCC 

In [373]:
# Décommenter pour charger les données d'apprentissage :

#Data = load_data(210000,6000,10000,5000)

In [166]:
# Chargement du réseau, sans apprentissage, car les poids sont enregistrés

# Décommenter pour effectuer la partie d'apprentissage et de test

# Résultat de l'apprentissage : acc 75% / precision sensitivity specifity 75% / MCC = 0.51

# paramètres :

#true_y = Data[5]
#training = Data[0]
#y = Data[1]
batch_size=96
epochs=25
verbose1 = 1
verbose2 = 1
#validation = Data[2]
#val_y = Data[3]
#testing = Data[4]
matrixsize11 = 6
nbfilter11 = 24
matrixsize12 = 7
nbfilter12 = 4
matrixsize21 = 6
nbfilter21 = 24
matrixsize22 = 7
nbfilter22 = 4
nbfilter1 = 64
kernelsize = 7
nbfilters2 = 64
kernel_size2 = 7
Dense1 = 64
Dense2 = 512
Dense3 = 512
Dense4 = 128
Dense5 = 0

k = matrixsize11
# init_weights
I = np.eye(k)
M = np.diag(np.ones(k-1),1) + np.diag(np.ones(k-1),-1) + np.eye(k)
I2 = np.zeros((k,k))
M2 = np.zeros((k,k))
for j in range(k):
    I2[:,j] = I[:,k-j-1]
    M2[:,j] = M[:,k-j-1]        
W = np.zeros((k,k,1,nbfilter11))
W[:,:,0,0] = I
W[:,:,0,1] = I2
W[:,:,0,2] = M
W[:,:,0,3] = M2
for j in range(4,nbfilter11):
    W[:,:,0,j] = np.random.randn(k,k)*0.2


k2 = matrixsize12
I = np.eye(k2)
M = np.diag(np.ones(k2-1),1) + np.diag(np.ones(k2-1),-1) + np.eye(k2)
I2=np.zeros((k2,k2))
M2=np.zeros((k2,k2))
for j in range(k2):
    I2[:,j] = I[:,k2-j-1]
    M2[:,j] = M[:,k2-j-1]   

Z = np.zeros((k2,k2,nbfilter11,nbfilter12))

for u in range(nbfilter12):
    Z[:,:,u,0] = I
    Z[:,:,u,1] = I2
    Z[:,:,u,2] = M
    Z[:,:,u,3] = M2
    for p in range(4,nbfilter12):
        Z[:,:,u,p]=np.random.randn(k2,k2)*0.3            

c2d1_input = keras.Input(shape=(36,36,1))
cnn2d1 = Conv2D(filters = nbfilter11, kernel_size=(k,k), padding='valid', input_shape=(36,36,1),strides=(1,1),weights=[W,np.zeros(nbfilter11)])(c2d1_input)
cnn2d1 = AveragePooling2D(pool_size=(3,3))(cnn2d1)
cnn2d1 = Conv2D(filters = nbfilter12, kernel_size = (k2, k2),strides=(1,1),padding='valid',weights=[Z,np.zeros(nbfilter12)])(cnn2d1)
cnn2d1 = Dropout(0.2)(cnn2d1)
cnn2d1 = Flatten()(cnn2d1)

c1d1_input = keras.Input(shape=(36,4))
cnn1d1 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d1_input)
cnn1d1 = MaxPooling1D(pool_size=8)(cnn1d1)
cnn1d1 = Dropout(0.2)(cnn1d1)
cnn1d1 = Flatten()(cnn1d1)

c1d2_input = keras.Input(shape=(36,4))
cnn1d2 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d2_input)
cnn1d2 = MaxPooling1D(pool_size=8)(cnn1d2)
cnn1d2 = Dropout(0.2)(cnn1d2)
cnn1d2 = Flatten()(cnn1d2)

c1d3_input = keras.Input(shape=(36,4))
cnn1d3 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d3_input)
cnn1d3 = MaxPooling1D(pool_size=8)(cnn1d3)
cnn1d3 = Dropout(0.2)(cnn1d3)
cnn1d3 = Flatten()(cnn1d3)

c1d4_input = keras.Input(shape=(36,4))
cnn1d4 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d4_input)
cnn1d4 = MaxPooling1D(pool_size=8)(cnn1d4)
cnn1d4 = Dropout(0.2)(cnn1d4)
cnn1d4 = Flatten()(cnn1d4)

model21 = keras.layers.concatenate([cnn1d1,cnn1d2])
model21 = Dense(int(Dense2/2),kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model21)
model21 = BatchNormalization()(model21)
model21 = Activation('relu')(model21)
model21 = Dropout(0.3)(model21)

model22 = keras.layers.concatenate([cnn1d3,cnn1d4])
model22 = Dense(int(Dense2/2),kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model22)
model22 = BatchNormalization()(model22)
model22 = Activation('relu')(model22)
model22 = Dropout(0.3)(model22)

model2 = keras.layers.concatenate([model21,model22])
model2 = Dense(Dense2,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model2)
model2 = BatchNormalization()(model2)
model2 = Activation('relu')(model2)
model2 = Dropout(0.3)(model2)

model1 = Dense(Dense1,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(cnn2d1)
model1 = BatchNormalization()(model1)
model1 = Activation('relu')(model1)

model = keras.layers.concatenate([model1,model2])
model = Dense(Dense3,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
model = Activation('relu')(model)
model = Dense(Dense4,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
model = Activation('relu')(model)

if Dense5>0:
    model = Dense(Dense5,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)

model = Dense(2,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
interaction_output = Activation('softmax')(model)

interaction = Model(inputs=[c2d1_input,c1d1_input,c1d2_input,c1d3_input,c1d4_input],outputs=[interaction_output])

interaction.compile(loss='categorical_crossentropy', optimizer='rmsprop')

#earlystopper = EarlyStopping(monitor='val_loss', patience=3, verbose = verbose1)

#print('model training')
#interaction.fit(training, y, batch_size, epochs, verbose = verbose2, validation_data=(validation, val_y), callbacks=[earlystopper])

# test   
#print('predicting')
#predictions = interaction.predict(testing)[:,1]
#print(predictions)
#for i,nulll in enumerate(predictions):
#    predictions[i] = round(predictions[i])
#print(predictions,true_y)
#perfs = calculate_performance(len(predictions), predictions, true_y)
#print("batch_size : ", batch_size)
#print("epochs : ",epochs)
#print("acc : ", perfs[0])
#print("precision : ", perfs[1])
#print("sensitivity : ", perfs[2])
#print("specificity : ", perfs[3])
#print("MCC : ", perfs[4])

model training


In [167]:
# Chargement des poids du réseau : 

interaction.load_weights("model_rna-rna_deep_livrablev2sansproba.h5")

### Test benchmark

#### Enregistrement des données benchmark non apprises par le réseau
benchmark = Data[6]
bdd = benchmark
ind_positifs = np.argwhere(bdd[:,4]==1)[:,0]
bdd = bdd[ind_positifs]
l = len(bdd)
writer = csv.writer(open("genomes/benchmark_interactionsv2.csv", 'w',newline=''),delimiter="\t")
for i in range(l):
    writer.writerow(bdd[i,:])

In [253]:
# Chargement des données benchmark
datatest = pd.read_csv("genomes/benchmark_interactionsv2.csv", sep = "\t",header=None)
datatest = np.array(datatest)
sequence1=[]
sequence2=[]
longueur1=[]
longueur2=[]
l = len(datatest)
for i in range(l):
    data = datatest[i][0].split(' ')
    data[0] = data[0].split('[')
    data[0][1] = data[0][1].split('\n')
    data[1] = data[1].split('\n')
    seq1=data[0][1][0][1:-1]
    seq2=data[1][0][1:-1]
    len1=int(data[2])
    len2=int(data[3])
    sequence1.append(seq1)
    sequence2.append(seq2)
    longueur1.append(len1)
    longueur2.append(len2)
sequence1=np.array(sequence1,dtype=None,ndmin=2)
sequence2=np.array(sequence2,dtype=None,ndmin=2)
longueur1=np.array(longueur1,dtype=None,ndmin=2)
longueur2=np.array(longueur2,dtype=None,ndmin=2)

bdd = np.zeros((len(datatest),4))
bdd = np.array(bdd,dtype='<U206',copy=True)
bdd[:,0]=sequence1
bdd[:,1]=sequence2
bdd[:,2]=longueur1
bdd[:,3]=longueur2

In [32]:
def remplissage(matrice,f):
    matrice_out = np.zeros((1,36,36,1))
    matrice_out[0,:f,:f,0] = matrice[0,:f,:f,0]
    return matrice_out
def remplissage_seq(matrice,f):
    matrice_out = np.zeros((1,36,4))
    matrice_out[0,:f] = matrice[0,:f]
    return matrice_out

In [370]:
# f longueur des sites à tester sur chaque séquence
f = 25
# s vitesse de glissement de la fenêtre de test le long des deux séquences
s = 12
# facteur_seuil afin de décider quelles prédictions sont considérées comme des vraies interactions
facteur_seuil = 0.95
#calcul des MCC sur la base de données benchmark constituées de 2433 paires de séquences dont les milieux interagissent ensemble
mcc = np.zeros(len(bdd))

for r in range(len(bdd)):
    
    seq1 = bdd[r,0]
    seq2 = reverse(bdd[r,1])
    dim = len(seq1)
    dim2 = len(seq2)
    matrice = np.zeros((1,dim,dim2,1)) 
    matrice2 = np.zeros((1,dim,4))
    matrice3 = np.zeros((1,dim2,4))


    for j in range(dim):
        for k in range(dim2):
            if (seq1[j]=='a' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='a'):
                matrice[0,j,k,0] = 1
            elif (seq1[j]=='g' and seq2[k]=='c') or (seq1[j]=='c' and seq2[k]=='g'):
                matrice[0,j,k,0] = 1
            elif (seq1[j]=='g' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='g'):
                matrice[0,j,k,0] = 1
    for j in range(dim):
        if seq1[j]=='a':
            matrice2[0,j,0] = 1
        elif seq1[j]=='u':
            matrice2[0,j,1] = 1
        elif seq1[j]=='g':
            matrice2[0,j,2] = 1
        elif seq1[j]=='c':
            matrice2[0,j,3] = 1
    for j in range(dim2):
        if seq2[j]=='a':
            matrice3[0,j,0] = 1
        elif seq2[j]=='u':
            matrice3[0,j,1] = 1
        elif seq2[j]=='g':
            matrice3[0,j,2] = 1
        elif seq2[j]=='c':
            matrice3[0,j,3] = 1
    u = int((dim-f+1)/s)
    u2 = int((dim2-f+1)/s)
    resultats = np.zeros((u,u2))

    for i in range(u):
        for j in range(u2):        
            test = []
            test.append(remplissage(matrice[:,i*s:i*s+36,j:j+36,:],f))
            test.append(remplissage_seq(matrice2[:,i*s:i*s+36],f))
            rev = np.zeros((1,36,4))
            for q in range(36):
                rev[:,36-q-1,:] = matrice2[:,q,:]
            test.append(remplissage_seq(rev,f))
            test.append(remplissage_seq(matrice3[:,j*s:j*s+36],f))
            rev2 = np.zeros((1,36,4))
            for q in range(36):
                rev2[:,36-q-1,:] = matrice3[:,q,:]
            test.append(remplissage_seq(rev2,f))

            resultats[i,j] = interaction.predict(test)[:,1]
    # seuil :
    seuil = np.max(resultats)*facteur_seuil
    pred = np.zeros(dim+dim2)
    for i in range(u):
        for j in range(u2):
            if resultats[i,j]>seuil:
                pred[i*s:i*s+f] = np.ones(f)
                pred[j*s:j*s+f] = np.ones(f)
    label = np.zeros(dim+dim2)
    label[85:85+int(bdd[r,2])]=np.ones(int(bdd[r,2]))
    label[dim+85:dim+85+int(bdd[r,3])]=np.ones(int(bdd[r,3]))
    # MCC : 
    mcc[r] = calculate_performance(dim+dim2,pred,label)[4]

In [375]:
np.mean(mcc)

0.06807466038244833